# Document Layout Aware Processing and Retrieval Augmented Generation.

This notebook was tested on a SageMaker Studio Notebook `Data Science 3.0` kernel and  `ml.t3.xlarge` instance.

---
---

## Contents

1. [Objective](#Objective)
1. [Background](#Background-(Problem-Description-and-Approach))
1. [Document Extraction](#Document-Extraction)
1. [Document Processing](#Document-Processing)
1. [Document Chunking](#Document-Chunking)
1. [Indexing](#Indexing)
1. [RAG](#RAG)
1. [CleanUp](#CleanUp)
1. [Conclusion](#Conclusion)

---

## Objective

This example notebook guides you through the process of utilizing Amazon Textract's layout feature. This feature allows you to extract content from your document while maintaining its layout and reading format. Amazon Textract Layout feature is able to detect the following sections:
- Titles
- Headers
- Sub-headers
- Text
- Tables
- Figures
- List 
- Footers
- Page Numbers
- Key-Value pairs

Here is a snippet of Textract Layout feature on a page of Amazon Sustainability report using the Textract Console UI:
![Alt text]("https://github.com/aws-samples/layout-aware-document-processing-and-retrieval-augmented-generation/blob/main/images/amazonsus2022.jpg")

The [Amazon Textract Textractor Library](https://aws-samples.github.io/amazon-textract-textractor/index.html) is a library that seamlessly works with Textract features to aid in document processing. You can start by checking out the [examples in the documentation.](https://aws-samples.github.io/amazon-textract-textractor/notebooks/layout_analysis_for_text_linearization.html)
This notebook utilizes the Textractor library to interact with Amazon Textract and interpret its response. It enriches the extracted document text with XML tags to delineate sections, facilitating layout-aware chunking and document indexing into a Vector Database (DB). This process aims to enhance Retrieval Augmented Generation (RAG) performance.

---

## Background (Problem Description and Approach)

- **Problem statement**: 
RAG serves as a technique aimed at enhancing the effectiveness of Large Language Models (LLMs) on lengthy textual content. While widely adopted, implementing RAG necessitates initial processing to extract and segment text into meaningful chunks, especially challenging for intricate assets like PDFs. Many document parsing approaches overlook layout semantics or use simplistic methods like fixed window carving, lacking awareness of document structure or elements. This can disrupt contextual continuity and diminish the performance of RAG systems. An optimal RAG input pipeline would intelligently divide PDF texts into vectorized segments aligned with layout and content semantics, preserving informational integrity for the LLM. In essence, a context-aware parsing phase is pivotal for enabling RAG techniques to realize their full potential, particularly when handling extensive or intricate documents.

- **Our approach**: 

<img src="images/txt layout-Page-2.jpg"/> 

1. Upload multi-page document to Amazon S3.
2. Call Amazon Textract Start Document Analysis api call to extract Document Text including Layout and Tables. The response provides structured text aligned with the original document formatting and the pandas tables of each table detected in the document.
3. Enrich this extracted text further with XML tags indicating semantic sections, adding contextual metadata through the Textractor library.
4. The textrcat library extracts tables in plain text, maintaining their original layout. However, for improved processing and manipulation, it's advisable to convert them to CSV format. This method replaces the plain text tables with their CSV counterparts obtained from Textract's table feature.
5.  In this approach, the extracted text is segmented based on document title sections, the highest hierarchy level in a document. Each subsection within the title section is then chunked according to a maximum word threshold. Below outlines our approach to handling the chunking of subsection elements.:

    - **Tables:** Tables are chunked row by row until the maximum number of alphanumeric words is reached. For each table chunk, the column headers are added to the table along with the table header, typically the sentence or paragraph preceding the table in the document. This ensures that the information of the table is retained in each chunk.
    
    <img src="images/table chunkers.png"/> 
    
    - **List:** Chunking lists found in documents can be challenging. Naive chunking methods often split list items by sentence or newline characters. However, this approach presents issues as only the first list chunk typically contains the list title, which provides essential information about the list items. Consequently, subsequent list chunks become obsolete. In this notebook, lists are chunked based on their individual list items. Additionally, the header of the list is appended to each list chunk to ensure that the information of the list is preserved in each chunk.
    <img src="images/list chunker.png"/>
    
    - **Section and subsection:** The structure of a document can generally be categorized into titles, sections, and paragraphs. A paragraph is typically the smallest unit of a document that conveys information independently, particularly within the context of a section or subsection header. In this method, text sections are chunked based on paragraphs, and the section header is added to each paragraph chunk (as well as tables and lists) within that section of the document.
    <img src="images/text chunks.png"/>
    
6. Metadata is appended to each respective chunk during indexing, encompassing:
    - The entire CSV tables detected within the chunk.
    - The section header ID associated with the chunk.
    - The section title ID linked to the chunk.
    
    When retrieving a passage based on hybrid search (combining semantic and text matching), there's flexibility in the amount of content forwarded to the LLM. Some queries may necessitate additional information, allowing users to choose whether to send the corresponding chunk subsection or title section based on the specific use case.

 *Some chunk may exceed the fixed word count threshold due to preserving paragraphs and dealing with complex tables. 

**Prerequisite:**
- [Amazon Bedrock model access](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)
- [Deploy Embedding and Text Generation Large Language Models with SageMaker JumpStart](https://docs.aws.amazon.com/sagemaker/latest/dg/jumpstart-foundation-models-use.html)
- [Create OpenSearch Domian](https://docs.aws.amazon.com/opensearch-service/latest/developerguide/createupdatedomains.html). This solution uses **IAM** as master user for fine-grained access control.


## Step 1: Setup

Install required packages

In [ ]:
!pip install --force-reinstall amazon-textract-textractor==1.7.1
!pip install inflect
!pip install langchain
!pip install requests-aws4auth
!pip install opensearch-py
!pip install anthropic

In [7]:
import os
from PIL import Image
import pandas as pd
import re
import json
from textractor import Textractor
from textractor.visualizers.entitylist import EntityList
from textractor.data.constants import TextractFeatures
import io
import inflect
import boto3
s3=boto3.client("s3")
from botocore.config import Config
config = Config(
    read_timeout=600, 
    retries = dict(
        max_attempts = 5 
    )
)
from anthropic import Anthropic
client = Anthropic()
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
bedrock_runtime = boto3.client(service_name='bedrock-runtime',region_name='us-east-1',config=config)

Utility functions are provided for text generation and embedding generation using select models from Amazon SageMaker Jumpstart and Amazon Bedrock. These models were chosen arbitrarily, but you have the flexibility to customize them by using models of your preference available on Bedrock, SageMaker JumpStart, or HuggingFace.\
**Change the placeholders for sagemaker endpoint names for the respective models below.**

In [190]:
def _get_emb_(passage, model):
    if "titan" in model:
        response = bedrock_runtime.invoke_model(body=json.dumps({"inputText":passage}),
                                    modelId="amazon.titan-embed-text-v1", 
                                    accept="application/json", 
                                    contentType="application/json")

        response_body = json.loads(response.get('body').read())
        embedding=response_body['embedding']
    elif "all-mini-lm" in model:
        payload = {'text_inputs': [passage]}
        payload = json.dumps(payload).encode('utf-8')

        response = SAGEMAKER.invoke_endpoint(EndpointName="SAGEMAKER JUMPSTART ALL MINI LM V6 ENDPOINT", 
                                                    ContentType='application/json',  
                                                    Body=payload)

        model_predictions = json.loads(response['Body'].read())
        embedding = model_predictions['embedding'][0]
    return embedding


from langchain.llms.sagemaker_endpoint import LLMContentHandler, SagemakerEndpoint
parameters ={
        "max_new_tokens": 100,       
        "return_full_text": False,
        "temperature": 0.1
}

class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        input_str = json.dumps({"inputs": prompt, "parameters": model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json[0]["generated_text"]


content_handler = ContentHandler()
mixtral_instruct = SagemakerEndpoint(
    endpoint_name="SAGEMAKER JUMPSATRT MIXTRAL ENDPOINT",
    region_name="us-east-1",
    model_kwargs=parameters,
    content_handler=content_handler,
)

## Document Extraction
We employ the Amazon Sustainability report as an example document. Using the textractor library, we trigger the Amazon Textract `start document analysis` API to initiate an asynchronous process for extracting document text and identifying additional elements like document layout and tables.\
Change **BUCKET** placeholder to your bucket name in Amazon S3.


In [ ]:
#Upload file to s3
BUCKET="BUCKET"
!aws s3 cp amazon-2014-10k.pdf s3://{BUCKET}/finance/

In [19]:
extractor = Textractor(region_name="us-east-1")
file=f"s3://{BUCKET}/finance/amazon-2014-10k.pdf"
doc_id=file_name = os.path.basename(file)

document = extractor.start_document_analysis(
    file_source=file,
    features=[TextractFeatures.LAYOUT,TextractFeatures.TABLES],
    # client_request_token=doc_id,
    save_image=False,
)

By leveraging the Textractor linearization function, we enhance the extracted content with XML tags while concealing certain page sections such as headers, footers, and non-essential images.

We opt to tag tables, lists, title sections, and sub-sections to facilitate the efficient identification and chunking of these document elements.

In [20]:
from textractor.data.text_linearization_config import TextLinearizationConfig

config = TextLinearizationConfig(
    hide_figure_layout=True,
    title_prefix="<titles><<title>><title>",
    title_suffix="</title><</title>>",
    hide_header_layout=True,
    section_header_prefix="<headers><<header>><header>",
    section_header_suffix="</header><</header>>",
    table_prefix="<tables><table>",
    table_suffix="</table>",
    list_layout_prefix="<<list>><list>",
    list_layout_suffix="</list><</list>>",
    hide_footer_layout=True,
    hide_page_num_layout=True,
)

print(document.pages[28].get_text(config=config))



<headers><<header>><header>Free Cash Flow Less Principal Repayments of Finance Leases and Financing Obligations </header><</header>>

Free cash flow less principal repayments of finance leases and financing obligations is free cash flow reduced by "Principal repayments of finance leases" and "Principal repayments of financing obligations." Principal repayments of finance leases and financing obligations approximates the actual payments of cash for our finance leases and financing obligations. The following is a reconciliation of free cash flow less principal repayments of finance leases and financing obligations to the most comparable GAAP cash flow measure, "Net cash provided by (used in) operating activities," for 2022 and 2023 (in millions): 



<tables><table>	Year Ended December 31, 	
	2022 	2023 
Net cash provided by (used in) operating activities 	$ 46,752 	$ 84,946 
Purchases of property and equipment, net of proceeds from sales and incentives 	(58,321) 	(48,133) 
Free cash f

## Document Processing

This code snippet comprises a Python function `split_list_items_` and a script segment that processes a document containing tables and text, converting tables into CSV format and maintaining the document structure with text and tables.

The function `split_list_items_` takes a string as input, likely representing a document with nested lists marked by specific XML tags. It parses this string, extracting items and handling nested lists appropriately. The function then returns a list containing the extracted items.

The script segment following the function processes each page of the document. It identifies tables, converts them to CSV format, and wraps them with XML tags for identification. If lists are present in the document, the script utilizes the `split_list_items_` function to handle them. The processed content is stored in dictionaries for further use.

This script segment efficiently manages document content, ensuring tables are properly formatted while preserving the document's structure with text and lists. It serves to handle data extraction and processing tasks involving documents with mixed content types.

In [21]:
def split_list_items_(items):
    """
    Splits the given string into a list of items, handling nested lists.

    Parameters:
    items (str): The input string containing items and possibly nested lists.

    Returns:
    list: A list containing the items extracted from the input string.
    """
    parts = re.split("(<<list>><list>|</list><</list>>)", items)  
    output = []

    inside_list = False
    list_item = ""

    for p in parts:
        if p == "<<list>><list>":
            inside_list = True    
            list_item=p
        elif p == "</list><</list>>":
            inside_list = False
            list_item += p
            output.append(list_item)
            list_item = "" 
        elif inside_list:
            list_item += p.strip()
        else:
            output.extend(p.split('\n'))
    return output


import io
"""
This script processes a document containing tables and text. It converts the tables into CSV format 
and wraps them with XML tags for easy identification. The document structure with text and tables is maintained.
"""
csv_seperator="|"
document_holder={}
table_page={}
count=0

# Loop through each page in the document
for ids,page in enumerate(document.pages):
    table_count=len([word for word in page.get_text(config=config).split() if "<tables><table>" in word]) # get the number of table in the extracted document page by header we set earlier
    assert table_count==len(page.tables) # check that number of tables per page is same as *tables extracted by textract TABLE feature
    content=page.get_text(config=config).split("<tables>")
    document_holder[ids]=[]    
    for idx,item in enumerate(content):
        if "<table>" in item:               
            df0=document.tables[count].to_pandas(use_columns=False).to_csv(header=False, index=None,sep=csv_seperator)
            row_count=len([x for x in df0.split("\n") if x]) #Check the number of rows in the parsed table to determine how to read the table headers. if table row count is 1 then headers is obviously at 0 else headers may or may not be at 0
            #Check if the first row in the csv is empty headers
            if row_count>1:
                if not all(value.strip() == '' for value in df0.split("\n")[0].split(csv_seperator)): 
                    row_count=1
            df=pd.read_csv(io.StringIO(df0), sep=csv_seperator, 
                           header=0 if row_count==1 else 1, keep_default_na=False) # read table with appropiate column headers
            df.rename(columns=lambda x: '' if x.startswith('Unnamed:') else x, inplace=True) 
            table=df.to_csv(index=None, sep=csv_seperator)
            if ids in table_page:
                table_page[ids].append(table)
            else:
                table_page[ids]=[table]
            # Extract table data and remaining content
            pattern = re.compile(r'<table>(.*?)(</table>)', re.DOTALL) 
            data=item
            table_match = re.search(pattern, data)
            table_data = table_match.group(1) if table_match else '' 
            remaining_content = data[table_match.end():] if table_match else data
            content[idx]=f"<<table>><table>{table}</table><</table>>" ## attach xml tags to differentiate table from other text
            count+=1
            # Check for list items in remaining content
            if "<<list>>" in remaining_content:
                output=split_list_items_(remaining_content)
                output=[x.strip() for x in output if x.strip()]
                document_holder[ids].extend([content[idx]]+output)           
            else:
                document_holder[ids].extend([content[idx]]+[x.strip() for x in remaining_content.split('\n') if x.strip()]) # split other text by new line to be independent items in the python list.
        else:   
            # Check for list items and tables in remaining content
            if "<<list>>" in item and "<table>" not in item:   
                output=split_list_items_(item)
                output=[x.strip() for x in output if x.strip()]
                document_holder[ids].extend(output)
            else:
                document_holder[ids].extend([x.strip() for x in item.split("\n") if x.strip()])

Here we first flatten a nested list into a single list and then join its elements using newline characters. Subsequently, the string is split into segments based on the `<titles>` tag (split by title section hierarchy), generating a list of sub-section segments. Following this, the function `sub_header_content_splitta` is defined to process a string, splitting it by XML tags and extracting text segments, excluding segments containing specific XML tags such as `<header>`, `<list>`, or `<table>`. This function takes a string as input, applies a regular expression pattern to split it by XML tags, and iterates through the resulting segments to filter out those containing the specified XML tags. The extracted text segments are then returned as a list. 

In [22]:
# # Flatten the nested list document_holder into a single list and Join the flattened list by "\n"
flattened_list = [item for sublist in document_holder.values() for item in sublist]
result = "\n".join( flattened_list)
header_split=result.split("<titles>")

def sub_header_content_splitta(string):   
    """
    Splits the input string by XML tags and returns a list containing the segments of text,
    excluding segments containing specific XML tags such as "<header>", "<list>", or "<table>".

    Parameters:
    string (str): The input string to be processed.

    Returns:
    list: A list containing the segments of text extracted from the input string.
    """ 
    pattern = re.compile(r'<<[^>]+>>')
    segments = re.split(pattern, string)
    result = []
    for segment in segments:
        if segment.strip():
            if "<header>" not in segment and "<list>" not in segment and  "<table>" not in segment:
                segment=[x.strip() for x in segment.split('\n') if x.strip()]
                result.extend(segment)
            else:
                result.append(segment)
    return result


## Document Chunking
This cell iterates through the document per title section and chunks content within each sub-sections in the following manner:
- It uses number of words as chunking threshold.
- It looks for the different xml tags to identify the different document content types. 
    - Iterating through the various sub-section within a section title identified by the **sub-section header** and only chunking contents within each sub-section. No chunk include multiple subsection content even if the max words threshold has not been met.
    - If a table xml tag is found, it checks if there is a sentence before that table (the heueristics employed here is that the sentence before a table is usually the table header) and use it as table headers. It then splits table by rows until desired chunk is achieved and appends the corresponding section header to the table chunk.
    - If a list is found, split list by items until desired chunk is achieved. Employ same heuristics as above and append list headers to all list chunk.
    - For other text, it chunks by paragraphs and appends each sub-section header to the corresponding chunks.
- A dicionary containing each complete sub-section is also stored to be used as metadata during indexing.
- The complete table found in each chunk is also stored for metadata purposes.

In [272]:
import re
import pandas as pd
from io import StringIO

max_words = 200
chunks = {}
table_header_dict={} 
chunk_header_mapping={}

# iterate through each title section
for title_ids, items in enumerate(header_split):
    title_chunks = []
    current_chunk = []
    num_words = 0   
    table_header_dict[title_ids]={}
    chunk_header_mapping[title_ids]={}
    chunk_counter=0
    for item_ids,item in enumerate(items.split('<headers>')): #headers
        lines=sub_header_content_splitta(item)             
        SECTION_HEADER=None 
        num_words = 0  
        for ids_line,line in enumerate(lines): #header lines  
            
            if line.strip():
                if "<header>" in line:   
                    SECTION_HEADER=re.findall(r'<header>(.*?)</header>', line)[0].strip()
                    line=SECTION_HEADER    
                    first_header_portion=True
                next_num_words = num_words + len(re.findall(r'\w+', line))  

                if  "<table>" not in line and "<list>" not in line:
                    if next_num_words > max_words and "".join(current_chunk).strip()!=SECTION_HEADER and current_chunk:
                        if SECTION_HEADER :
                            if first_header_portion:
                                first_header_portion=False                                            
                            else:
                                current_chunk.insert(0, SECTION_HEADER.strip())                       

                        title_chunks.append(current_chunk)                  
                        chunk_header_mapping[title_ids][chunk_counter]=lines
                        # print(num_words,current_chunk)
                        current_chunk = []
                        num_words = 0 
                        chunk_counter+=1

                    current_chunk.append(line)    
                    num_words += len(re.findall(r'\w+', line))

                """
                Goal is to segment out table items and chunks intelligently.
                We chunk the table by rows and for each chunk of the table we append the table column headers
                and table headers if any. This way we preserve the table information across each chunks.
                This will help improve semantic search where all the chunks relating to a table would be in the 
                top k=n response giving the LLM mcomplet information on the table.
                """

                if "<table>" in line:
                    # Get table header which is usually line before table in document              
                    line_index=lines.index(line)
                    if line_index!=0 and "<table>" not in lines[line_index-1] and "<list>" not in lines[line_index-1]: #Check if table is first item on the page, then they wont be a header (header may be included it table) and also if table is the the last item in the list
                        header=lines[line_index-1].replace("<header>","").replace("</header>","")
                    else:
                        header=""                   
                    
                    table = line.split("<table>")[-1].split("</table>")[0] # get table from demarcators     
                    df=pd.read_csv(io.StringIO(table), sep=csv_seperator, keep_default_na=False)
                    df.rename(columns=lambda x: '' if x.startswith('Unnamed:') else x, inplace=True)                    
                    table_chunks = []
                    curr_chunk = [df.columns.to_list()] #start current chunk with table column names    
                    words=len(re.findall(r'\w+', str(current_chunk)+" "+str(curr_chunk)))  
                    # Iterate through the rows in the table
                    for row in df.itertuples():
                        curr_chunk.append(row)         
                        words+=len(re.findall(r'\w+', str(row)))
                        if words > max_words:                        
                            if [x for x in table_header_dict[title_ids] if chunk_counter == x]:
                                table_header_dict[title_ids][chunk_counter].extend([header]+[table])
                            else:
                                table_header_dict[title_ids][chunk_counter]=[header]+[table]                            
                            table_chunks.append("\n".join(["|".join(str(x) for x in curr_chunk[0])] + ["|".join(str(x) for x in r) for r in curr_chunk[1:]])) #join chunk lines together to for a csv 
                            tab_chunk="\n".join(["|".join(str(x) for x in curr_chunk[0])] + ["|".join(str(x) for x in r) for r in curr_chunk[1:]]) #join chunk lines together to for a csv
                            words = len(re.findall(r'\w+', str(curr_chunk[0]))) # set word count to word length of column header names
                            if header: #If header  attach header to table                         
                                if current_chunk and current_chunk[-1].strip().lower()==header.strip().lower(): #check if header is in the chunk and remove to avoid duplicacy of header in chunk                        
                                    current_chunk.pop(-1)
                                # Append section header to table
                                if SECTION_HEADER and SECTION_HEADER.lower().strip() != header.lower().strip():
                                    if first_header_portion:
                                        first_header_portion=False
                                    else:
                                        current_chunk.insert(0, SECTION_HEADER.strip())                             
                                current_chunk.extend([header.strip()+':' if not header.strip().endswith(':') else header.strip() ]+[tab_chunk]) #enrich table header with ':'
                                title_chunks.append(current_chunk)                           
                        
                            else:
                                if SECTION_HEADER:
                                    if first_header_portion:
                                        first_header_portion=False
                                    else:
                                        current_chunk.insert(0, SECTION_HEADER.strip())                                
                                current_chunk.extend([tab_chunk])
                                title_chunks.append(current_chunk)                        
                            chunk_header_mapping[title_ids][chunk_counter]=lines
                            chunk_counter+=1
                            num_words=0
                            current_chunk=[]
                            curr_chunk = [curr_chunk[0]]
                    
                    if curr_chunk != [df.columns.to_list()] and lines.index(line) == len(lines)-1: #if table chunk still remaining and table is last item in page append as last chunk
                        table_chunks.append("\n".join(["|".join(str(x) for x in curr_chunk[0])] + ["|".join(str(x) for x in r) for r in curr_chunk[1:]]))
                        tab_chunk="\n".join(["|".join(str(x) for x in curr_chunk[0])] + ["|".join(str(x) for x in r) for r in curr_chunk[1:]])                        
                        if [x for x in table_header_dict[title_ids] if chunk_counter == x]:
                            table_header_dict[title_ids][chunk_counter].extend([header]+[table])
                        else:
                            table_header_dict[title_ids][chunk_counter]=[header]+[table]   
                        
                        if header: 
                            if current_chunk and current_chunk[-1].strip().lower()==header.strip().lower():#check if header is in the chunk and remove to avoid duplicacy of header in chunk
                                current_chunk.pop(-1) 
                            if SECTION_HEADER and SECTION_HEADER.lower().strip() != header.lower().strip():
                                if first_header_portion:
                                    first_header_portion=False
                                else:
                                    current_chunk.insert(0, SECTION_HEADER.strip())                          
                            current_chunk.extend([header.strip()+':' if not header.strip().endswith(':') else header.strip() ]+[tab_chunk])
                            title_chunks.append(current_chunk)                   
                        else:
                            if SECTION_HEADER:
                                if first_header_portion:
                                    first_header_portion=False
                                else:
                                    current_chunk.insert(0, SECTION_HEADER.strip())                            
                            current_chunk.extend([tab_chunk])
                            title_chunks.append(current_chunk)             
                        chunk_header_mapping[title_ids][chunk_counter]=lines
                        chunk_counter+=1
                        num_words=0
                        current_chunk=[]
                    elif curr_chunk != [df.columns.to_list()] and lines.index(line) != len(lines)-1: #if table is not last item in page and max word threshold is not reached, send no next loop
                        table_chunks.append("\n".join(["|".join(str(x) for x in curr_chunk[0])] + ["|".join(str(x) for x in r) for r in curr_chunk[1:]]))
                        tab_chunk="\n".join(["|".join(str(x) for x in curr_chunk[0])] + ["|".join(str(x) for x in r) for r in curr_chunk[1:]])
                        
                        if [x for x in table_header_dict[title_ids] if chunk_counter == x]:
                            table_header_dict[title_ids][chunk_counter].extend([header]+[table])
                        else:
                            table_header_dict[title_ids][chunk_counter]=[header]+[table]                         
                        if header:               
                            if current_chunk and current_chunk[-1].strip().lower()==header.strip().lower():#check if header is in the chunk and remove to avoid duplicacy of header in chunk
                                current_chunk.pop(-1) 
                            current_chunk.extend([header.strip()+':' if not header.strip().endswith(':') else header.strip() ]+[tab_chunk])
                        else:
                            current_chunk.extend([tab_chunk])                  
                        num_words=words
                     

                """
                Goal is to segment out list items and chunk intelligently.
                We chunk each list by items in the list and 
                for each list chunk we append the list header to the chunk to preserve the information of the list across chunks.
                This would boost retrieval process where question pertaining to a list will have all list chunks within
                the topK=n responses.
                """

                if "<list>" in line:
                    # Get list header which is usually line before list in document
                    line_index=lines.index(line)
                    if line_index!=0 and "<table>" not in lines[line_index-1] and "<list>" not in lines[line_index-1]: #Check if table or list is the previous item on the page, then they wont be a header
                        header=lines[line_index-1].replace("<header>","").replace("</header>","")
                    else:
                        header=""           
                    list_pattern = re.compile(r'<list>(.*?)(?:</list>|$)', re.DOTALL)   ## Grab all list contents within the list xml tags        
                    list_match = re.search(list_pattern, line)
                    list_ = list_match.group(1)
                    list_lines=list_.split("\n")                

                    curr_chunk = []  
                    words=len(re.findall(r'\w+', str(current_chunk)))  #start word count from any existing chunk
                    # Iterate through the items in the list
                    for lyst_item in list_lines:
                        curr_chunk.append(lyst_item)         
                        words+=len(re.findall(r'\w+', lyst_item)) 
                        if words >= max_words: #  
                            words=0     
                            list_chunk="\n".join(curr_chunk)
                            if header: # attach list header                       
                                if current_chunk and current_chunk[-1].strip().lower()==header.strip().lower():#check if header is in the chunk and remove to avoid duplicacy of header in chunk                        
                                    current_chunk.pop(-1)  
                                # Append section content header to list
                                if SECTION_HEADER and SECTION_HEADER.lower().strip() != header.lower().strip():
                                    if first_header_portion:
                                        first_header_portion=False
                                    else:
                                        current_chunk.insert(0, SECTION_HEADER.strip())
                                    
                                current_chunk.extend([header.strip()+':' if not header.strip().endswith(':') else header.strip() ]+[list_chunk]) 
                                title_chunks.append(current_chunk)                          
                         
                            else:
                                if SECTION_HEADER:
                                    if first_header_portion:
                                        first_header_portion=False
                                    else:
                                        current_chunk.insert(0, SECTION_HEADER.strip())
                                    
                                current_chunk.extend([list_chunk])
                                title_chunks.append(current_chunk)                            
                            chunk_header_mapping[title_ids][chunk_counter]=lines
                            chunk_counter+=1
                            num_words=0
                            current_chunk=[]
                            curr_chunk = []
                    if curr_chunk  and lines.index(line) == len(lines)-1: #if list chunk still remaining and list is last item in page append as last chunk
                        list_chunk="\n".join(curr_chunk)
                        if header: 
                            if current_chunk and current_chunk[-1].strip().lower()==header.strip().lower(): #check if header is in the chunk and remove to avoid duplicacy of header in chunk
                                current_chunk.pop(-1)                            
                            if SECTION_HEADER and SECTION_HEADER.lower().strip() != header.lower().strip():
                                if first_header_portion:
                                    first_header_portion=False
                                else:
                                    current_chunk.insert(0, SECTION_HEADER.strip())                   
                            current_chunk.extend([header.strip()+':' if not header.strip().endswith(':') else header.strip() ]+[list_chunk])
                            title_chunks.append(current_chunk)                        
                        else:
                            if SECTION_HEADER:
                                if first_header_portion:
                                    first_header_portion=False
                                else:
                                    current_chunk.insert(0, SECTION_HEADER.strip())                   
                            current_chunk.extend([list_chunk])
                            title_chunks.append(current_chunk)                     
                        chunk_header_mapping[title_ids][chunk_counter]=lines
                        chunk_counter+=1
                        num_words=0
                        current_chunk=[]
                    elif curr_chunk and lines.index(line) != len(lines)-1: #if list is not last item in page and max word threshold is not reached, send to next loop          
                        list_chunk="\n".join(curr_chunk)
                        if header:               
                            if current_chunk and current_chunk[-1].strip().lower()==header.strip().lower():#check if header is in the chunk and remove to avoid duplicacy of header in chunk
                                current_chunk.pop(-1) 
                            current_chunk.extend([header.strip()+':' if not header.strip().endswith(':') else header.strip() ]+[list_chunk])
                        else:
                            current_chunk.extend([list_chunk])                  
                        num_words=words


        if current_chunk and "".join(current_chunk).strip()!=SECTION_HEADER:
            if SECTION_HEADER:
                if first_header_portion:
                    first_header_portion=False
                else:
                    current_chunk.insert(0, SECTION_HEADER.strip())         
            title_chunks.append(current_chunk)
            chunk_header_mapping[title_ids][chunk_counter]=lines
            current_chunk=[]
            chunk_counter+=1
    if current_chunk:
        title_chunks.append(current_chunk) 
        chunk_header_mapping[title_ids][chunk_counter]=lines
    chunks[title_ids] = title_chunks

In [ ]:
# Print chunks per title section
for i, chunk in enumerate(chunks[2], start=1):
    print(f'Chunk {i}:')
    for item in chunk:
        print(item)
    print('\n')

In [106]:
# List of title header sections document was split into
for x in chunk_header_mapping:
    title_pattern = re.compile(r'<title>(.*?)(?:</title>|$)', re.DOTALL)       
    title_match = re.search(title_pattern, chunk_header_mapping[x][0][0])
    title_ = title_match.group(1) if title_match else ""
    print(title_, end='\n')


FORM 10-K 
FORM 10-K For the Fiscal Year Ended December 31, 2023 
PART II 
Item 7. Management's Discussion and Analysis of Financial Condition and Results of Operations 
Table of Contents 
INDEX TO CONSOLIDATED FINANCIAL STATEMENTS 
Report of Independent Registered Public Accounting Firm 
AMAZON.COM, INC. CONSOLIDATED STATEMENTS OF CASH FLOWS (in millions) 
CONSOLIDATED STATEMENTS OF OPERATIONS 
AMAZON.COM, INC. CONSOLIDATED STATEMENTS OF COMPREHENSIVE INCOME (LOSS) (in millions) 
CONSOLIDATED BALANCE SHEETS 
AMAZON.COM, INC. CONSOLIDATED STATEMENTS OF STOCKHOLDERS' EQUITY (in millions) 
AMAZON.COM, INC. NOTES TO CONSOLIDATED FINANCIAL STATEMENTS 
Note 2 - FINANCIAL INSTRUMENTS 
Report of Independent Registered Public Accounting Firm 
PART III 
PART IV 
SIGNATURES 
AMAZON.COM, INC. GLOBAL RESTRICTED STOCK UNIT AWARD AGREEMENT 
ACCEPTANCE AND ACKNOWLEDGMENT 
GLOBAL RESTRICTED STOCK UNIT AWARD AGREEMENT 
LIST OF SIGNIFICANT SUBSIDIARIES 
Consent of Independent Registered Public Accounti

Upload section contents (title and headers for each chunk) to s3

In [140]:
with open (f"{doc_id}.json", "w") as f:
    json.dump(chunk_header_mapping,f)
s3.upload_file(f"{doc_id}.json", BUCKET, f"{doc_id}.json")

## Indexing


Here's a sample script for indexing document chunks into an Amazon OpenSearch Service Domain, alternatively [Amazon OpenSearch Serverless](https://aws.amazon.com/blogs/big-data/introducing-the-vector-engine-for-amazon-opensearch-serverless-now-in-preview/) can be used.

This code block establishes an index within an Amazon OpenSearch Service (Provisioned Capacity) and proceeds to index the document chunks. The index mapping incorporates metadata fields such as document name, complete chunk tables, header section IDs, and title section IDs. These section IDs can be utilized to retrieve section content stored elsewhere (e.g., S3 or a different OpenSearch Service Domain Index) for matching passages to retrieve additional information to be passed to the LLM.

We utilize an embedding model to generate embeddings and subsequently index them. The provided options in this implementation are `all-mini-lm v6` and `Amazon titan Embedding`. If utilizing the former, deploy the SageMaker endpoint and provide the name in the utility function in cell 3 above.

**Note:** Certain chunks may exceed the threshold set for chunking in the previous cells due to the way tables are chunked by row and section paragraph sizes. This might result in a token limit exceed error for certain embedding models.

Ensure to replace the **domain_endpoint** variable with the Amazon OpenSearch Service domain you created in your account.

In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth

"""
This script demonstrates indexing documents into an Amazon OpenSearch Service domain using AWS Identity and Access Management (IAM) for authentication.
"""
# Embedding Model
model="all-mini-lm" # other option "titan", you can also use any other model in Bedrock or SageMaker Jumpstart or HuggingFace

domain_endpoint = "AMAZON SAGEMAKER OPENSEARCH SERVICE DOMAIN ENDPOINT" #replace wit your OpenSearch Service domain endpoint
service = 'es'
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, "us-east-1", service, session_token=credentials.token)
os_ = OpenSearch(
    hosts = [{'host': domain_endpoint, 'port': 443}],
    http_auth = awsauth,
    use_ssl = True,
    verify_certs = True,
    timeout=120,        
    # http_compress = True, # enables gzip compression for request bodies
    connection_class = RequestsHttpConnection
)

# Sample Opensearch domain index mapping
mapping = {
  'settings': {
    'index': {  
      'knn': True,
      "knn.algo_param.ef_search": 100,            
    }
      },

      'mappings': {  
        'properties': {
          'embedding': {
            'type': 'knn_vector', 
            'dimension':384 if "all-mini-lm" in model else (1536 if "titan" in model else None), #change as per sequence length of Embedding Model
            "method": {
              "name": "hnsw",       
              "space_type": "innerproduct",
              "engine": "nmslib",
              "parameters": {
                 "ef_construction": 256,
                 "m":  48
               }
            }
          },

          'passage': {
            'type': 'text'
          },

          'doc_id': {
            'type': 'keyword'
          },
        
          'table': {
            'type': 'text'
          },
        
          'title_headers': {
            'type': 'text'
          },
          'section_header_ids': {
            'type': 'text'
          },
          'section_title_ids': {
            'type': 'text'
          },

        }
      }
    }

domain_index = f"experiment0"    

if not os_.indices.exists(index=domain_index):        
    os_.indices.create(index=domain_index, body=mapping)
    # Verify that the index has been created
    if os_.indices.exists(index=domain_index):
        print(f"Index {domain_index} created successfully.")
    else:
        print(f"Failed to create index '{domain_index}'.")
else:
    print(f'{domain_index} Index already exists!')

i = 1

SAGEMAKER=boto3.client('sagemaker-runtime')
for ids, chunkks in chunks.items(): # Iterate through the page title chunks 
    title_pattern = re.compile(r'<title>(.*?)(?:</title>|$)', re.DOTALL)       
    title_match = re.search(title_pattern, str(chunkks))
    title = title_match.group(1) if title_match else ""
    for chunk_ids,chunk in enumerate(chunkks): # iterating through section header chunks  
        passage_chunk="\n".join(chunk).replace("<title>","").replace("</title>","")        
        embedding=_get_emb_(passage_chunk, model)       
        table=[]
        if ids in table_header_dict:
            if [x for x in table_header_dict[ids] if x ==chunk_ids]:                
                table="\n".join(table_header_dict[ids][chunk_ids])        
        
        
        documentt = { 
            'doc_id':doc_id, #doc name   
            'passage': passage_chunk,
            'embedding': embedding,
            'table':table,
            "title_headers":title.strip(),
            "section_header_ids":chunk_ids, #Store id of the header section
            "section_title_ids":ids #Store id of the title section
        }
   
        try:
            response = os_.index(index=domain_index, body=documentt)
            i += 1
            # Check the response to see if the indexing was successful
            if response["result"] == "created":
                print(f"Document indexed successfully with ID: {response['_id']}")
            else:
                print("Failed to index document.")
        except RequestError as e:
            logging.error(f"Error indexing document to index '{domain_index}': {e}")
        

In [269]:
from opensearchpy import Transport

"""
This script initializes a transport client for Amazon OpenSearch Service and creates a search pipeline with a normalization-processor. 
The normalization technique in the processor is set to min_max, and the combination technique is set to arithmetic_mean.
"""
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, "us-east-1", service, session_token=credentials.token)
transport = Transport(
   hosts = [{'host': domain_endpoint, 'port': 443}],
    http_auth = awsauth,
    use_ssl = True,
    verify_certs = True,
    timeout=120,        
    # http_compress = True, # enables gzip compression for request bodies
    connection_class = RequestsHttpConnection
)
pipeline_name="norm-pipeline01"
pipeline_definition = {
    "description": "Post-processor for hybrid search",
    "phase_results_processors": [
        {
            "normalization-processor": {
                "normalization": {
                    "technique": "min_max"
                },
                "combination": {
                    "technique": "arithmetic_mean",
                    
                }
            }
        }
    ]
}

# Send the PUT request to create the search pipeline
response = transport.perform_request("PUT", f"/_search/pipeline/{pipeline_name}", body=pipeline_definition)



# RAG

Opensearch Hybrid Search combining semantic and keyword search. We return topK=3 results.

In [ ]:
question="What are the total gross lease liabilities for 2022 and 2023?"
embedding=_get_emb_(question, model)
# Define the search query
query = {   
    'size': 3,
    "_source": {
    "exclude": [
      "embedding"
    ]
  },
    "query": {
    "hybrid": {
      "queries": [
          {
          "match": {
              "passage": question
            }
        },
          {
          "knn": {
          "embedding": {
            "vector": embedding,
            "k": 3
          }
        }
        },
        
        
      ]
    }
  }
}
# Send the search request with the defined query and search pipeline
response = transport.perform_request("GET", f"/{domain_index}/_search?search_pipeline={pipeline_name}", body=query)
response

The capability to select hierarchical information of interest from the retrieved response is based on the corresponding matched passage chunk. This allows for flexibility in determining the amount of information provided to the LLM, accommodating queries that benefit from access to full section information.

This approach entails storing chunk hierarchical sections and titles in an S3 bucket. If additional information beyond the passage chunk in the OpenSearch index is necessary, it retrieves the corresponding sections based on the indexed matching IDs associated with the chunk.

In [238]:
def read_file_from_s3(bucket_name, key, section_content, title_id=None,section_id=None):
    """
    Read a file from an S3 bucket and extract specific sections based on given parameters.
    Parameters:
        bucket_name (str): The name of the S3 bucket.
        key (str): The key (path) of the file in the S3 bucket.
        section_content (str): Specifies the type of section content to extract.
            Possible values: "section_header" or "section_title".
        title_id (str or int, optional): The ID of the title. Required if `section_content` is "section_header".
        section_id (str or int, optional): The ID of the section. Required if `section_content` is "section_header".

    Returns:
        str or None: The extracted section content as a string. Returns None if there's an error.
    """
    try:   
        response = s3.get_object(Bucket=bucket_name, Key=key)        
        file_content = response['Body'].read().decode('utf-8')
        file_content=json.loads(file_content)
        if section_content=="section_header":
            passage=file_content[str(title_id)][str(section_id)]
        elif section_content=="section_title":
            passage=[item for sublist in file_content[str(title_id)].values() for item in sublist]
        return "\n".join(passage)
    except Exception as e:
        print(f"Error reading {key} from S3 bucket {bucket_name}:", e)
        return None

class InvalidContentError(Exception):
    pass

# Extract relevant information from the search response
def content_extraction_os_(response:str, table:bool, section_content:str):
    """
    Extracts content from the OpenSearch response based on specified parameters.

    Parameters:
    response (dict): The response from OpenSearch containing search results.
    table (bool): A boolean indicating whether to include table content.
    section_content (str): The type of content to extract. Allowed values are 'passage', 'section_header', or 'section_title'.

    Returns:
    tuple: A tuple containing concatenated passages and tables.
    """
    allowed_values = {"passage", "section_header", "section_title"}  # Define allowed values
    if section_content not in allowed_values:
        raise InvalidContentError(f"Invalid content type '{section_content}'. Allowed values are {', '.join(allowed_values)}.")
    
    res=response['hits']['hits']
    score = [str(x['_score']) for x in res]  #retrieval score    
    title_names = [x['_source']['title_headers'] for x in res] #doc page number of chunks
    doc_name = [x['_source']['doc_id'] for x in res] # doc names
    header_ids = [x['_source']['section_header_ids'] for x in res] # section header id
    title_ids=[x['_source']['section_title_ids'] for x in res] # section title id
    tables=""
    
    if section_content=="passage":
        passage = [x['_source']["passage"] for x in res] #retrieved passages, here you can choose to retrieve the  complete section header or title instead of the chunk passage
        tables=[x['_source']['table'] for x in res] # tables in the corresponding chunk
    else:
        passage=[]
        for x in range(len(title_ids)):
            passage.append(read_file_from_s3(BUCKET, f"{doc_id}.json",section_content,title_ids[x],header_ids[x]))
        passage=set(passage)      
    p = inflect.engine()
    ## Concatenate passages and tables to use in prompt template 
    passages=""
    tab=""
    for  ids,text in enumerate(passage):
        passages+=f"<{p.ordinal(ids+1)}_passage>\n{text}\n</{p.ordinal(ids+1)}_passage>\n"
    if table and tables:
        for  ids,text in enumerate(tables):            
            tab+=f"<{p.ordinal(ids+1)}_passage_table>\n{text}\n</{p.ordinal(ids+1)}_passage_table>\n"  #Table can be coupled with passage chunks to provide more information.
    return passages, tab

In [252]:
passages,tab=content_extraction_os_(response, False, "passage")
print(passages,tab)

<1st_passage>
Note 4 - LEASES
|December 31, 2023 ||
0||Operating Leases |Finance Leases |Total 
1||||
2|Gross lease liabilities |$ 90,777 |$ 14,106 |$ 104,883 
3|Less: imputed interest |(15,138) |(1,997) |(17,135) 
4|Present value of lease liabilities |75,639 |12,109 |87,748 
5|Less: current portion of lease liabilities |(8,419) |(2,032) |(10,451) 
6|Total long-term lease liabilities |$ 67,220 |$ 10,077 |$ 77,297 
</1st_passage>
<2nd_passage>
Note 4 - LEASES
Lease cost recognized in our consolidated statements of operations is summarized as follows (in millions):
|Year Ended December 31, ||
7|Total lease cost |$ 19,085 |$ 17,157 |$ 18,918 
Other information about lease amounts recognized in our consolidated financial statements is as follows:
|December 31, 2022 |December 31, 2023 
0|||
1|Weighted-average remaining lease term - operating leases |11.6 years |11.3 years 
2|Weighted-average remaining lease term - finance leases |10.3 years |11.9 years 
3|Weighted-average discount rate - op

## Bedrock Anthropic LLM Inference

In [222]:
prompt_template=f"""You are a helpful, obedient and truthful financial assistance.

<document>
{passages}
</document>          

<additional_information>
{tab}
</additional_information>

<instructions>
When providing your response based on the document:
1. Understand the question to know what is being asked of you.
2. Review the entire document provided and check if it contains relevant information to answer the question. Only pay attention to passages with relevant information.
3. If the document is sufficient to answer the question, provide a comprehensive answer ENTIRELY based on the document provided. DO NOT make up answers not present in the document.
4. If the answer is not available in the document, say so.
</instructions>

Question: {question}
if able to answer:
    Format you response as:
    Your Answer
    <source>document or additional info tag(s) containing the relevant info</source>"""
prompt=f"\n\nHuman:{prompt_template}\n\nAssistant: Based on the document,"

print(f' Size of prompt token is {client.count_tokens(prompt)}')

 Size of prompt token is 1056


In [223]:
txt_model='anthropic.claude-v2'
inference_modifier = {'max_tokens_to_sample':300, 
                      "temperature":0.1,
                      # "top_k":250,
                      # "top_p":1,               
                     }
llm = Bedrock(model_id=txt_model, client=bedrock_runtime, model_kwargs = inference_modifier,
              streaming=True,  # Toggle this to turn streaming on or off
              callbacks=[StreamingStdOutCallbackHandler() ])

response = llm(prompt)

 the total gross lease liabilities for 2022 and 2023 are:

2022: $99,292 million
<source>2nd_passage</source>

2023: $104,883 million  
<source>1st_passage</source>

## SageMaker JumpStart Mixtral 8x7b Inference

In [231]:
prompt=f"""<s><<SYS>>[INST]
You are a helpful, obedient and truthful assistant.

Here is a document:
####
{passages}
####

Here is additional information:
####
{tab}
####
<</SYS>>
When providing your response based on the provided document:
1. Understand the question to know what is being asked of you.
2. Review the entire document provided and check if it contains relevant information to answer the question. Only pay attention to sections with relevant information.
3. If the document is sufficient to answer the question, provide a comprehensive answer ENTIRELY based on the document provided. DO NOT make up answers not present in the document.
4. If the answer is not available in the document, say so.

Question: {question}
if able to answer:
    Format your response as:
    Your Answer
    <source>document or additional info tags containing the relevant info</source>[/INST]
Answer: According to the document provided,"""
parameters = { "max_new_tokens": 300,       
        "return_full_text": False,
        "temperature": 0.1}
mixtral_instruct.model_kwargs = parameters
print(mixtral_instruct(prompt))

 the total gross lease liabilities for 2022 were $99,292 million and for 2023 were $104,883 million.

<source>
<1st_passage>
|December 31, 2023 ||
0||Operating Leases |Finance Leases |Total 
1||||
2|Gross lease liabilities |$ 90,777 |$ 14,106 |$ 104,883
</1st_passage>

<2nd_passage>
|December 31, 2022 |December 31, 2023 
0|||
1|Gross lease liabilities |$ 81,273 |$ 18,019 |$ 99,292
</2nd_passage>
</source>


# CleanUp

To avoid incurring future charges, delete the resources you created including any sagemaker endpoints and or opensearch service domain no longer in use.

In [ ]:
# Sample code to delete sagemaker endpoint

sagemaker = boto3.client('sagemaker')
# Specify the name of the domain to delete
endpoint_name = 'your_endpoint_name_here'
try:
    response = sagemaker.delete_endpoint(
        EndpointName=endpoint_name
    )
    print(f"SageMaker endpoint '{endpoint_name}' deleted successfully.")
except sagemaker.exceptions.ClientError as e:
    error_code = e.response['Error']['Code']
    if error_code == 'ValidationException' and 'Could not find endpoint' in e.response['Error']['Message']:
        print(f"SageMaker endpoint '{endpoint_name}' not found.")
    else:
        print(f"Failed to delete SageMaker endpoint '{endpoint_name}': {e}")
except Exception as e:
    print(f"An error occurred: {e}")   


Do make sure that you are no longer using you opensearch domain before you commence to delete it. [Here is a link on how to delete your OpenSearch Service domain](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/opensearch/client/delete_domain.html)

# Conclusion

This notebook showcases the extraction of content from a document while maintaining its layout structure. Additionally, we processed and chunked the document, ensuring the integrity of the information was preserved. Furthermore, we indexed these chunks and associated hierarchical metadata information, offering flexibility in information retrieval. 

Finally, we conducted a RAG query and generated contextual answers.